<a id="inicio"></a>
<img src="./figs/barra_uclm_esiiab.png" alt="Banner UCLM - ESIIAB" align="right">

<br><br><br>
<h1><font color="#B30033" size=5>Intelligent Systems - Course 2021-2022</font></h1>



<h1><font color="#B30033" size=5>Assignment 4: Metaheuristics</font></h1>


<br><br>
<div style="text-align: left">
<font color="#4E70BE" size=3>Lecturers:</font><br>
<ul>
  <li><font color="#4E70BE" size=3>Juan Carlos Alfaro Jiménez</font><br></li>
  <li><font color="#4E70BE" size=3>Guillermo Tomás Fernández Martín</font><br></li>
  <li><font color="#4E70BE" size=3>José Antonio Gámez Martín</font><br></li>
  <li><font color="#4E70BE" size=3>Ismael García Varea</font><br></li>
  <li><font color="#4E70BE" size=3>Luis González Naharro</font><br></li>
  <li><font color="#4E70BE" size=3>Jesús Martínez Gómez</font><br></li>    
</ul>
</div>

<br>

## Introduction

Metaheuristic algorithms are often used to obtain good solutions for difficult problems. By difficult problems we mean: problems expressed as a *blackbox*, that is, problems for which its internal structure or definition is unknown; problems whose representation or evaluation is so complex that it is impossible to solve it using other algorithmic techniques; and problems for which even knowing its internal structure the number of configurations (potential solutions) is so large that the exact solution would be too costly even for small size instances.

In this assignment, the two types of metaheuristic algorithms studied in class, namely *local search* and *genetic algorithms*, will be implemented. They will be used to solve a specific problem related to the search problem implemented in Assignment 2. Apart from studying the behavior of the algorithms and the influence that different parameters exert in the obtained solutions, different modifications can be proposed and implemented with the purpose of improving the accuracy (better solutions) and the efficiency (less evaluations, less time) of the implemented algorithms.

## Problem description
For the sake of completeness we also include the description of the problem in this assignment.

The maze is a grid of size N x M formed by a set of cells, some of which can be occupied by walls, which cannot be crossed. The rest of the cells will be empty and they will represent the free space. For now, the robot can only move horizontally or vertically. In addition, we can have cells with garbage, which must be cleaned by our agent, a vacuum cleaner robot.

The objective of our robot is to clean the whole area as fast as possible. In other words: **find the shortest path to find all the garbage cells in the environment**. In order to implement our robot we have to take into account that: 
- The robot can start in a random cell of the map.
- The robot can move horizontally or vertically in the maze.
- The robot can not cross walls or go beyond the limits of the maze.
- The robot will have to clean all the garbage cells found in the map, which will be automatically cleaned as soon as the robot arrives to those cells.
- For now, all the movements of the robot will have a cost of 1.
- The search will finish once all garbage cells in the map have been cleaned.

Then, in this assignment the problem to solve consists on finding the order in which all the garbages should be cleaned (visited), starting from the initial position of the robot. Therefore every possible ordering of the garbages will be a possible solution to our problem, namely **configuration** in the following. As stated in Assignment 2, the A$^*$ search algorithm is the best option to find the shortest path between two points in a maze if we want to guarantee the optimality of the search process, which in addition is more efficient than UniformCost search (or BreadthFirst in our case, since all actions have a cost of 1). Then, in order to find the shortest path to find all the garbage cells in the environment we will apply the A$^*$ search iteratively from the initial position of the robot to the first garbage in the configuration, then to the second one, and so on until the last one.  

## Provided code

In the following we provide you some of the clases, implemented in `Python` that will help you to develop this assignment. 

First, we will import the necessary classes we need from the Python libraries

In [635]:
%config IPCompleter.greedy=True


In [636]:
import math
import copy
import time
from queue import PriorityQueue 

Next, we will import some custom functions from the file `utils.py`. You don't need to modify those functions for the code to work, but feel free to have a look at them if you are curious. This code is identical to the one provided in Assignment 2.

In [637]:
from utils import *

Finally, we will import some third party libraries. We will use those to display the problem in a graphical environment. In order to do that, we will use the magic functions from jupyter to install the library from inside the notebook

In [638]:
!pip install ipythonblocks

You should consider upgrading via the 'C:\Users\49427234\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [639]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output

In order to complete the requested search algorithms, we provide you some fundamental classes: 

#### Class `Action`
This class provides the **representation of the actions** that will be performed by the robot. You don't have to modify the code of this class. The possible actions will be: "UP", "DOWN", "RIGHT", "LEFT". This class is identical to the one provided in Assignment 2 and, for the moment, the cost of all actions is kept at the value of 1.0.

In [640]:
class Action:
    #actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, move):
        self.move = move

    def __str__(self):
        return f"({self.move})"

    def getCost(self):
        return 1.0

#### Class `State`. 
This class provides the **representation of a state** in the search space. In this problem, a state is defined by the position of the robot. Note that the maze itself does not need to be part of the state given that it does not change during the search, i.e. walls are fixed during the search. You don't have to modify the code of this class.

The class `State` has an `applyAction` method that, given a valid `Action`, returns a new `State` with the action applied. This class is almost identical to the one provided in Assignment 2.

In [641]:
class State:

    def __init__(self, pos):
        self.pos = pos

    # equals method. Returns true if the states are the same. 
    # Used for the hash table comparison, compares if both states are equal
    def __eq__(self, state):
        return self.pos == state.pos 
    
    def __str__(self):
        return f"Position: {self.pos}\n"

    # hash method. Useful to index data structures that uses a hash function 
    #    to index elements, i.e. a Set()
    def __hash__(self):
        return int( math.pow(10,3)*self.pos[0] + self.pos[1])

    def applyAction(self, action):
        st = copy.deepcopy(self)
        
        if (action.move == "UP"):
            st.pos = (st.pos[0]-1,st.pos[1])
        elif (action.move == "DOWN"):
            st.pos = (st.pos[0]+1,st.pos[1])
        elif (action.move == "RIGHT"):
            st.pos = (st.pos[0],st.pos[1]+1)
        elif (action.move == "LEFT"):
            st.pos = (st.pos[0],st.pos[1]-1)
        else:
            print("\n*** ERROR ***: Action " + action + "  is not allowed .....\n")
            sys.exit()
        
        return st

#### Class `Node`. 
This class provides a **representation of a node** in the search tree/graph. It contains the state it represents, its parent node and the action taken to reach the current node. You don't have to modify this class. 

The class `Node` also has some methods to provide access to its attributes.

**The implementation is the same as in Assignment 2**. As a reminder:
- In addition to the `self.gGost` (cost to reach the node), two new attributes are defined: `self.hCost` and `self.fCost`, which are needed to store the heuristic and total cost of a specific node. These attributes should be conveniently assigned during the search process, accoding to the specific search strategy.
- Correspondingly two new methods are included: `getHCost(self)` and `getFCost(self)`
- The `__hash__` method is added, which is a special method useful to index data structures that uses a hash function to index elements.
- The `__lt__` method is added, which is a special method that defines the behaviour of the less-than operator (`<`) for objects of this class. In our case it is defined according to the value of the attribute `self.fCost`.

In [642]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state # Must be State Class
        self.parent = parent # Must be Node Class
        self.action = action # Must be Action Class
        self.depth = 0
        self.gCost, self.hCost, self.fCost = 0.0, 0.0, 0.0

    def __str__(self):
        return f"depth: {self.depth} fcost: {self.fCost}, ({self.gCost} + {self.hCost}), state: {self.state}"

    # equal method. Returns true if the states are the same. Used for the hash table comparison
    # compares if both states are equals
    def __eq__(self, other):
        if not isinstance(other, Node):
        # don't attempt to compare against unrelated types
            return NotImplemented
        return self.state == other.state

    # hash method. Useful to index data structures that uses a hash function 
    # to index elements, i.e. a Set()
    def __hash__(self):
        return int( math.pow(10,3)*self.pos[0] + self.pos[1])

    def __lt__(self, other):
        if not isinstance(other, Node):
            # don't attempt to compare against unrelated types
            return NotImplemented
        return self.fCost < other.fCost

    def getState(self):
        return self.state

    def getAction(self):
        return self.action
    
    def getParent(self):
        return self.parent

    def getDepth(self):
        return self.depth
    
    def getGCost(self):
        return self.gCost
    
    def getHCost(self):
        return self.hCost

    def getFCost(self):
        return self.fCost
    

#### Class `Problem`
As in Assignment 2, this class provides the **representation of the search problem**. It contains the size of the maze (`rows` and `cols`), the `initialState` and the `maze`. This class can read from a file an instance of the problem to solve, or it can generate a random instance using the size of the grid, the `seed`, the maximum number of divisions (walls) in the maze and the list of garbage cells. You don't have to modify this class. 

The class `Problem` also has some methods to provide access to the initial state and the possible actions.

For this assignment this class also provides the method `getConfigurationByNearestNeighbour(self)` that generates a configuration which is computed as the heuristic nearest neighbour from the position to the robot to the closest garbage (according to the heuristic), from this garbage to the next closest, and so on.

In [643]:
class Problem:

    actions = ["UP", "DOWN", "RIGHT", "LEFT"]

    def __init__(self, rows, cols, seed, maxDivisions, garbageCount, filename="", heuristic="manhattan"):

        if (filename != ""):
            self.rows, self.cols, self.maze = readProblemInstance(filename)
            print('Problem read with size',rows,'x',cols)
        else:
            self.rows = rows
            self.cols = cols
            self.maze = getProblemInstance(rows, cols, maxDivisions, garbageCount, seed)

        self.garbage = []
        self.heuristic = heuristic

        for r in range(rows):
            for c in range(cols):
                if self.maze[r][c] == 2:
                    init_point = (r,c)
                elif self.maze[r][c] == 3:
                    self.garbage.append((r,c))

        self.initialState = State(init_point)

    def getInitialState(self):
        return self.initialState

    def getActions(self):
        return self.actions
    
    # heuristic nearest neighbour configuration generation
    def getConfigurationByNearestNeighbour(self):
        conf = []

        garbageBis = list(range(len(self.garbage)))
        start = self.getInitialState()

        initPos = start.pos
        while (len(garbageBis)>1):
            d = math.inf
            ind = -1
            for i in range(len(garbageBis)):
                if self.heuristic=="euclidean":
                    d1 = euclidean_distance(initPos,self.garbage[garbageBis[i]])
                elif self.heuristic =="manhattan":
                    d1 = manhattan_distance(initPos,self.garbage[garbageBis[i]])
                else:
                    print("error: heuristic not implemented")
                    return None
                if d1<d:
                    d = d1
                    ind = i

            conf.append(garbageBis[ind])
            initPos = self.garbage[garbageBis[ind]]
            garbageBis.pop(ind)

        conf.append(garbageBis[0])    

        return conf

## Implementation
In the following we provide you some classes and pieces of code that you will have to complete as a part of this assignment.


#### Class `AStar`
This class implements a search class specifically defined to implement the A$^*$ search. The `AStar` class contains some attributes:
- The `problem` to solve.
- The `initial` and `goal` states to start from and reach to with the A$^*$ search.
- The name of the `heuristic` function to be used during the search: "euclidean" or "manhattan".
- The list of `open` nodes, i.e. nodes in the frontier. This list is undefined given that you have to use a priority queue data structure to implement that list. Remember that nodes will be inserted in order in this list, where the order is defined according to different criteria depending on each specific search algorithm. 
- The list of `closed` nodes to implement the graph search, which is implemented using a `set` data structure.
- The attributes to account for the number of generated and expanded nodes, as well as the maximum size of the open 'list' to know the maximum number of nodes stored simultaneously in memory. These attributes are helpful to estimate the time and memory complexity of the algorithms.

This class also provides three methods:
- `insertNode(self, node)`: this method implements the priorized insertion of a node into the list of open nodes. You have to implement this method. 
- `getSuccesors(self, node)`: this method implements the successors function and should return a list with all the valid successors of a given node. You have to implement this method.
- `doSearch(self)`: this method implements the graph search you have studied in class and also provides some statistics of the search process. You have to implement this method.

All these three methods have already implemented in Assignment 2, so we highly suggest to copy or adapt them for this assignment, where the goal is to go from the `start` state to the `goal` state.

In [644]:
class AStar: 

    def __init__(self, problem, start, goal, heuristic):
        self.problem = problem
        self.initial = start
        self.goal = goal
        self.heuristic = heuristic
        
        self.open = PriorityQueue()
        self.closed = set()

        self.generatedNodes = 0
        self.expandedNodes = 0
        
        self.solutionCost = 0
        self.solutionDepth = 0

        
        
    def insertNode(self, node):
        if self.heuristic=="euclidean":
            d = euclidean_distance(node.getState().pos, self.goal.pos)
        elif self.heuristic == "manhattan":
            d = manhattan_distance(node.getState().pos, self.goal.pos)
        else:
            print("\nError: heuristic must be euclidean or manhattan")
            sys.exit()
            
        node.fCost = node.gCost + d
        self.open.put((node.fCost, node))
        

    def getSuccesors(self, node):

        suc = []
        self.expandedNodes += 1
        
        for a in self.problem.getActions():
            action = Action(a)
            auxState = node.getState().applyAction(action)

            # We check if the state is inside the maze and not out of bounds
            if (auxState.pos[0] < self.problem.rows and 
                auxState.pos[0] >= 0 and
                auxState.pos[1] < self.problem.cols and
                auxState.pos[1] >= 0):
              cell = self.problem.maze[auxState.pos[0]][auxState.pos[1]]
              #print("Cell: " + str(cell))
              
              # We check if the new state will not lead to a wall
              if cell != 1:
                nodeAux = Node(node.state.applyAction(action), node, action)
                nodeAux.gCost = node.getGCost() + 1

                d = None
                if self.heuristic=="euclidean":
                    d = euclidean_distance(node.getState().pos, self.goal.pos)
                elif self.heuristic == "manhattan":
                    d = manhattan_distance(node.getState().pos, self.goal.pos)
                else:
                    print("\nError: heuristic must be euclidean or manhattan")
                    sys.exit()

                nodeAux.hCost = d
                nodeAux.fCost = nodeAux.gCost + nodeAux.hCost
                nodeAux.depth = node.getDepth() + 1
                suc.append(nodeAux)
                self.generatedNodes += 1
        
        return suc


    def doSearch(self):
        totalCost = 0
        
        current = Node(self.initial, None, None)

        actionSequence = []
     
        self.insertNode(current)
        self.generatedNodes += 1

        finish = False
        
        while (True):
            # No more nodes that we can open, then we exit the loop
            if (self.open.empty()):
                break

            # We get the node with the least cost (the cost (index) used depends on the algorithm)
            currentNode = self.open.get()[1]

            #self.exploredNodes += 1
            if currentNode.getState() == self.goal:
                self.closed.add(currentNode.getState())
                finish = True
                break

            # We check whether the node we have picked from open has been explored
            if currentNode.getState() not in self.closed: 
                suc = self.getSuccesors(currentNode)
                for s in suc:
                    self.insertNode(s)

            self.closed.add(currentNode.getState())
        
        if (finish):
            #print("Depth of the solution: " + str(currentNode.getDepth()))

            while (currentNode.getParent() != None):
                actionSequence.append(currentNode.getAction())
                currentNode = currentNode.getParent()
                self.solutionCost += 1

            #print("Generated nodes: " + str(self.generatedNodes))  
            #print("Expanded nodes: " + str(self.expandedNodes))  
            #print("Explored nodes: " + str(self.exploredNodes))
            #print("Solution cost: " + str(self.cost))
 
        else:
            print("No solution found ...")
        
        return actionSequence, self.generatedNodes, self.expandedNodes
    
    # getters
    
    def getGeneratedNodes(self):
        return self.generatedNodes
    
    def getExpandedNodes(self):
        return self.expandedNodes
    
    def getSolutionCost(self):
        return self.solutionCost
    
    def getSolutionDepth(self):
        return self.solutionDepth
    
    def getActionSequence(self):
        return self.actionSequence
    

#### Class `Evaluation`
This class defines and implements the functions needed to evaluate a specific configuration of our problem, which will be a **permutation** of the garbage indices $(0,..,N-1)$.

The `Evaluation` class contains the following attributes:
- The `problem` to solve.
- The name of the `heuristic` function to be used during the search. For example, "euclidean" or "manhattan".
- A boolean attribute, `partial`, that can be used if we want to implement a speed up mechanism which consists of not repeating A$^*$ searching processes already carried out. It is likely that if you do not implement this the search process will take quite a long time for some big problem instances, that is with a medium/large number of garbages.
- A `cache` of precomputed search paths. This dictionary stores the tuple (cost,path) of the   A$^*$ search for each (start,goal) pair of states, the first time is computed. Subsequent computations of the same search will be looked up in this dictionary. You also have to bear in mind that the reverse path, that is (goal,start), can also be included in the cache with the *correct* action sequence.

- The best configuration (`bestConf`), the best search path (`bestSolution`), and the associated cost (`bestCost`), found so far.
- The attributes to account for the number of `generated` and `expanded` nodes, as well as the number of searches performed (`numSearches`) and the number of configurations evaluated (`numConfs`).

This class also provides two methods:
- `evaluate(self, conf)`: this method implements the evaluation of a given configuration. To do that you have to carry out an A$^*$ search process iteratively from the initial position of the robot to the first garbage in the configuration, then to the second one, and so on until the last one is cleaned/visited. You have to implement this method.
- `printReport(self)`: this method can be used to print the result of the evaluation process along with some useful statistics. This method is provided.

It is important to note that the evaluation of a given configuration of length $N$ entails the execution of $N$ A$^*$ search process, whose solutions should be concatenated to provide the global search path obtained for that configuration.

In [645]:
class Evaluation:
    
    def __init__(self, problem, heuristic, partial):        
        self.problem = problem
        self.heuristic = heuristic
        self.partial = partial
        
        self.cache = {} # A dictionary of precomputed paths
        
        self.bestConf = [] # best configuration of the garbages
        self.bestSolution = [] # Concatenated sequence of actions
        self.bestCost = math.inf # best cost obtained as the sum of all AStar search costs
        
        self.generated = 0
        self.expanded = 0
        self.numSearches = 0
        self.numConfs = 0
          
    def evaluateConf(self, conf):
        
        cost = 0
        path_sol = []
        auxGenerated = 0
        auxExpanded = 0
        
        start = self.problem.getInitialState()
        
        # TODO
        # ...
            
        # desde la posición inicial hasta la primera caca, llamar al AStar
        # y así con todas las cacas

        debug = False
        if debug:
            for c in conf:
                print("Caca Nº" + str(c))
                print("Pos: " + str(self.problem.garbage[c]))

        for i in range(len(conf)):

            astar = None
            goal = State(self.problem.garbage[conf[i]])
            #print(str(goal))
            if i == 0:
                aStar = AStar(self.problem, start, goal, self.heuristic)
            else:
                newStart = State(self.problem.garbage[conf[i-1]])
                aStar = AStar(self.problem, newStart, goal, self.heuristic)

            actionSequence, auxGenerated, auxExpanded = aStar.doSearch()
            self.numSearches += 1
            #print(str(len(actionSequence)))
            i = 0
            for a in actionSequence:
                path_sol.append(a)
                i += 1
                #print(str(a))

        cost = len(path_sol)
        self.numConfs += 1

        if cost < self.bestCost:
            self.bestCost = cost
            self.bestConf = conf
            self.bestSolution = path_sol
            self.generated += auxGenerated
            self.expanded += auxExpanded
        #print("Cost: " + str(cost))

        # END-TODO
        
        return cost, path_sol
        
   
    # getters

    def getGeneratedNodes(self):
        return self.generated
    
    def getExpandedNodes(self):
        return self.expanded
    
    def getNumConfs(self):
        return self.numConfs
    
    def getNumSearches(self):
        return self.numSearches
    
    def getBestCost(self):
        return self.bestCost
    
    def getBestConf(self):
        return self.bestConf
    
    def getBestSolution(self):
        return self.bestSolution
    
    
    # print evaluation report
    # printing accumulated data over all the evaluations
    def printReport(self):
        
        print("\n\nTotal Generated nodes: ", self.generated)
        print("Total Expanded nodes: ", self.expanded)
        print("Total evaluated configurations: ", self.numConfs) 
        print("Total calls to AStar: ", self.numSearches)

        print("\nBest solution cost: ", self.bestCost)
        print("Best evaluated configuration: ", self.bestConf)
        print("Num actions: ", len(self.bestSolution))
        print("Sequence of actions:")
        for i in range(len(self.bestSolution)):
            print(self.bestSolution[i], end=" ")

#### Class `RandomSearch`

The `RandomSearch` class implements a random search along the search space of all possible configurations by simply generating and evaluating some randomly generated configurations. This class is provided as a baseline in order to compare results with your implementation.


In [646]:
class RandomSearch:
    
    def __init__(self, problem, numIt, heuristic, partial, seed):        
        self.partial = partial
        self.numIt = numIt
        self.problem = problem
        self.heuristic = heuristic
        random.seed(seed)

        self.evaluation = Evaluation(self.problem, self.heuristic, self.partial)
        
    
    def doSearch(self):

        conf = list(range(len(self.problem.garbage)))

        for it in range(self.numIt):
            conf = random.sample(conf, k=len(conf))
            cost, path_sol = self.evaluation.evaluateConf(conf)
     
        return self.evaluation  

#### Class `HillClimbing`

The `HillClimbing` class implements the local search algorithm studied in class. 

This class has the following attributes:
- `problem`: problem to solve.
- `numIt`: number of reinitializations of the search (for a RLS/ILS implementation). Note that if numtIt=1 the standard HillClimbing method is carried out.
- `start`: how to initilize the search. It can take two values: "random" or "informed". To implement the informed search you can make use of the method `getConfigurationByNearestNeighbour(heuristic)` defined in the class `problem`.
- `heuristic`: name of the heuristic function to be used in the A* search. It can be "euclidean" or "manhattan".
- `partial`: flag to apply the speed-up the algorithm using a lookup table of previous performed searches
- `seed`: seed to start random number generator.

This class implements the following methods:
- `swap(self, configuration, p1, p2)`: an operator to exchange the values of two positions in a configuration. You have to implement this method.
- `doSearch(self)`: this method implements the HillClimbing local search you have studied in class and also provides some statistics of the search process. You have to implement this method.

In [647]:
# HillClimbing class 
# problem: problem to solve
# numIt: number of time to reinit the search (for ILS)
# start: "random" or "informed"
# heuristic: name of the heuristic function to be used in the A* search
# partial: flag to apply a speed-up the algorithm using a lookup table of previous performed searches
# seed: seed to start random number generator


class HillClimbing:
    
    def __init__(self, problem, numIt, heuristic, partial, start, seed):        
        self.problem = problem
        self.numIt = numIt
        self.heuristic = heuristic
        self.partial = partial
        self.start = start
        random.seed(seed)
        self.numItems = len(problem.garbage)

        self.evaluation = Evaluation(self.problem, self.heuristic, self.partial)
        
    # swap the content of two conf positions - the input conf is modified  
    def swap(self, configuration, p1, p2):        
        # TODO
        # ...

        # END-TODO
        return configuration
    
    # runs the search process   
    def doSearch(self):
        
        # TODO
        # ...



        # END-TODO

        return self.evaluation



#### Class `GeneticAlgorithm`

The `GeneticAlgorithm` class implements a Genetic Algorithm global search algorithm studied in class. The individuals in our population will be pairs of (configuration, fitness).

This class has the following attributes:
- `problem`: problem to solve.
- `popSize`: size of the population.
- `maxGen`: maximun number of generations.
- `start`: how to initilize the search. It can take two values: "random" or "informed".
- `pc`: crossover probability.
- `pm`: mutation probability at individual level.
- `tournamentSize`: size of the tournament, if this selection method is used.
- `heuristic`: name of the heuristic function to be used in the A* search. It can be "euclidean" or "manhattan".
- `partial`: flag to apply the speed-up the algorithm using a lookup table of previous performed searches.
- `seed`: seed to start random number generator.

This class implements the following methods:
- `doSearch(self)`: this method implements the Genetic Algorithm global search you have studied in class and also provides some statistics of the search process. This method is already implemented for you.
- `getInitialPopulation(self)`: this method creates the initial population according to the `start` attribute value ("random" or "informed"). To use the informed initialization you can make use of the method `getConfigurationByNearestNeighbour(heuristic)` defined in the class `problem`. You have to implement this method.
- `getSelectedPopulation(self, population):` this method obtains a new population by using proportional to the fitness, range-based or tournament-based selection. You have to implement this method.
- `applyCrossover(self, population)`: this method applies a crossover operator over the given population. It might be useful to also implement a method to implement the crossover between two configurations, i.e. `crossover(self, parent1, parent2)`. You have to implement these methods.
- `applyMutation(self, population):` this method applies a mutation over the given population. We recommend to apply the mutation probability at individual level. You have to implement these methods.
- `getNewPopulation(self, pop1, pop2):` this method combines two populations and selects the individuals to be part of the next population according to one of the methods studied in classs. You have to implement this method.

In [648]:
# Genetic Algorithm class

# problem: problem to solve
# popSize: size of the population
# maxGen: maximun number of generations
# start: how to generate the initial population "random" o "informed"
# pc: crossover probability
# pm: mutation probability - at individual level
# tournamentSize: size of the tournament (if used)
# heuristic: name of the heuristic function to be used in the A* search
# partial: flag to apply a speed-up the algorithm using a lookup table of previous performed searches
# seed: seed to start random number generator

class GeneticAlgorithm:
    
    def __init__(self, problem, popSize, maxGen, start, pc, pm, tournamentSize, heuristic, partial, seed):        
        self.problem = problem
        self.popSize = popSize
        self.maxGen = maxGen
        self.pc = pc
        self.pm = pm
        self.tournamentSize = tournamentSize
        self.heuristic = heuristic
        self.partial = partial

        random.seed(seed)
        self.start = start
        self.numItems = len(problem.garbage)

        self.evaluation = Evaluation(self.problem, self.heuristic, self.partial)
        
    
    # create initial population
    # consider an individual as a tuple (conf, fitness)
    def getInitialPopulation(self):
        pop = []

        # TODO
        
        i = popSize
        while i > 0:
            individual = tuple()

            if self.start == "informed":
                conf = self.problem.getConfigurationByNearestNeighbour()
                fitness = self.evaluation.evaluateConf(conf)
                individual = tuple(conf, fitness)
            # elif self.start == "random":
            #    conf = self.

            pop.append(individual)
            i -= 1

        # END-TODO 
             
        return pop
            
        
    # obtain a new population    
    def getSelectedPopulation(self, population):
        pop = []
        # TODO
        # ...
        
        # END-TODO
        return pop
    
    
    # crossover between two confs   
    def crossover(self, parent1, parent2):
        offspring1 = copy.copy(parent1)
        offspring2 = copy.copy(parent2)
        
        #TODO
        # END TODO
        return offspring1, offspring2
    
    
    # apply crossover over the given population. Contiguous pairs are used
    def applyCrossover(self, population):
        
        pop = []
        # TODO
        # ...
        
        # END-TODO
        return pop
    
    # apply mutation over the given population 
    def applyMutation(self, population):
        # TODO
        # ...
        
        # END-TODO
        return population
            
            
    # combines two populations to get a new one      
        
    def getNewPopulation(self, pop1, pop2):
        pop = []
        # TODO
        
        # END TODO
        return pop
        
        
    # prints the average and best fitness of the population
    def print_pop_fitness(self, population, gen):
        fitness = [e[1] for e in population]
        print("Gen:", gen, " Max: ", max(fitness), " Average: ", sum(fitness)/len(fitness))
        
    # runs the search process
    def doSearch(self):
            
        population = self.getInitialPopulation()
        
        # main loop
        
        for g in range(self.maxGen):
        
            self.print_pop_fitness(population, g)
            
            pop_s = self.getSelectedPopulation(population)
            
            pop_c = self.applyCrossover(pop_s)
            
            pop_m = self.applyMutation(pop_c)
            
            for i in range(len(pop_m)):
                cost, path = self.evaluation.evaluateConf(pop_m[i][0])
                pop_m[i] = (pop_m[i][0], cost)
            
            population = self.getNewPopulation(population, pop_m)   
            
        return self.evaluation


#### The `main` function

Next, we provide you the `main` function that creates the problem and solves it using the search algorithm provided. This method should be used afterwards to carry out the experimentation to study the behaviour of the implemented algorithms for different values of the parameters provided (size of the maze, maximum number of walls, number of garbage cells, and algorithm).

In [649]:
# Problem parameters:
# row, cols, sedd, maxDivisions, garbageCount, configFile, heuristic
# Hill Climbing specific parameters:
# - numIt (default: 1): number of iterations (reinits)
# - partial (default: False): to speed up the search process with a look-up table
# - start (default: "random"): initialization method {"random", "informed"}

# Genetic Algorithm specific parameters:
# - partial (default: True): to speed up the search process with a look-up table
# - start (default: "random"): initialization method {"random", "informed"}
# - popSize (default: 50): population size
# - maxGen (default: 30): maximum number of generations
# - pc (default: 1.0): cross probability
# - pm (default: 0.2): mutation probability
# - tournamentSize (default: 2)

def main(rows, cols, seed, maxDivisions, garbageCount, algorithm, configFile="", heuristic="manhattan", numIter=1, partial=True, start="random",popSize=50, maxGen=30, pc=1.0, pm=0.2, tournamentSize=2):

    problem_instance = Problem(rows, cols, seed, maxDivisions, garbageCount, configFile, heuristic)
    print("Problem instance:")
    printMaze(problem_instance.maze)
    print("")

    search = None

    print("Search algorithm:", algorithm)

    if algorithm == "RandomSearch":
        search = RandomSearch(problem_instance, numIter, heuristic, partial, seed)
    elif algorithm == "HillClimbing":
        search = HillClimbing(problem_instance, numIter, heuristic, partial, start, seed)
    elif algorithm == "GeneticAlgorithm":
        search = GeneticAlgorithm(problem_instance, popSize, maxGen, start, pc, pm, tournamentSize, heuristic, partial, seed)
    else:
        raise Exception

    time_start = time.perf_counter()
    evaluation = search.doSearch()
    evaluation.printReport()
    path_sol = evaluation.getBestSolution()
    time_end = time.perf_counter()
    print("")
    print("Elapsed time: " + str(time_end - time_start) + " seconds")

    return path_sol, problem_instance

#### Test your code

Here you have a piece of code to test your implementation.

In [650]:
rows = 10
cols = 10
seed = 2021
maxDivisions = 2
garbageCount = 5
configFile = ""
heuristic = "manhattan"
partial = True
numIter= 5
start="informed"
popSize=50
maxGen=30
pc=1.0
pm=0.2
tournamentSize=2

algorithm = "RandomSearch"
#algorithm = "HillClimbing"
#algorithm = "GeneticAlgorithm"

path_sol, problem_instance = main(rows, cols, seed, maxDivisions, garbageCount, algorithm, configFile, heuristic, numIter, partial, start, popSize, maxGen, pc, pm, tournamentSize)





Problem instance:
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 1, 3, 0, 0]
[0, 0, 3, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 1, 0, 1, 1, 1, 1, 0, 3, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 2, 0, 0, 1, 0, 0, 0]

Search algorithm: RandomSearch


Total Generated nodes:  207
Total Expanded nodes:  60
Total evaluated configurations:  5
Total calls to AStar:  25

Best solution cost:  36
Best evaluated configuration:  [4, 3, 2, 0, 1]
Num actions:  36
Sequence of actions:
(LEFT) (LEFT) (LEFT) (UP) (RIGHT) (UP) (UP) (UP) (RIGHT) (RIGHT) (RIGHT) (RIGHT) (RIGHT) (RIGHT) (RIGHT) (UP) (LEFT) (LEFT) (LEFT) (LEFT) (LEFT) (UP) (LEFT) (UP) (LEFT) (LEFT) (UP) (UP) (RIGHT) (RIGHT) (RIGHT) (DOWN) (RIGHT) (RIGHT) (RIGHT) (RIGHT) 
Elapsed time: 0.07150750001892447 seconds


#### Printing the result

Here we provide you some code to display the maze and the path carried out by the robot (the green cell) to solve the instance of the problem. Walls are represented as black cells, and garbage with brown cells.

In [651]:
def render_maze(grid, maze, garbage):
    height, width = len(maze), len(maze[0])
    # Render maze
    for i in range(width):
        for j in range(height):
            grid[j,i] = (200,200,200) if maze[j][i] in [0,2,3] else (0,0,0)

    # Render garbage in maze
    for g in garbage:
        grid[g[0],g[1]] = (139,69,19)
        
def find_agent(maze, width, height):
    for i in range(width):
        for j in range(height):
            if (maze[j][i] == 2):
                return (i,j)

def render_path(path, maze, garbage):
    height, width = len(maze), len(maze[0])
    solution_grid = BlockGrid(width, height, fill=(200, 200, 200))
    
    movementDict = {'DOWN':(0,1), 'UP':(0,-1), 'LEFT':(-1,0), 'RIGHT':(1,0)}
    agentPos = find_agent(maze, width, height)
    garbageRender = copy.deepcopy(garbage)
    
    # Initial position rendering
    render_maze(solution_grid, maze, garbageRender)
    solution_grid[agentPos[1],agentPos[0]] = (0, 255, 0)
    solution_grid.show()
    clear_output(wait=True)
    time.sleep(0.1)
    
    for action in path:
        # Update agent position
        agentPos = (agentPos[0] + movementDict[action.move][0], agentPos[1] + movementDict[action.move][1])
        
        # Update garbage list
        if ((agentPos[1], agentPos[0]) in garbageRender):
            garbageRender.remove((agentPos[1], agentPos[0]))
        
        # Render maze
        render_maze(solution_grid, maze, garbageRender)
        
        # Render agent, and update its position and the garbage list
        solution_grid[agentPos[1],agentPos[0]] = (0,255,0)
        
        solution_grid.show()
        clear_output(wait=True)
        time.sleep(0.2)

You can easily try different instances of the problem just by changing the parameters when you call both the `main(...)` function and the `render_path(...)` function:

## Experimental results

Once the algorithms have been implemented, you must study their performance. In order to do that, you must compare the quality of the solutions obtained, as well as the number of expanded nodes for instances of different maze sizes, number of walls and number of garbage cells.

Please, use new cells to insert code to carry out the experimental results and study of the algorithms.